# Currying

**Currying** is a technique that transforms a single function that accepts multiple arguments, into a series of functions, each accepting a single argument. For example:

In [ ]:
#THIS IS THE SAME
def sum(a,b):
    return a+b
#THAN THIS
def sum(a):
    def inner_sum(b):
        return a+b
    return inner_sum

**`sum`** takes a single argument as input, and returns a function that takes a single input too, but this one returns the result of operating over the arguments.

The following example shows a similar case, but working with str transformations:

In [1]:
def converted_font_size(font_size):
    def inner_function(doc_type):
        if doc_type == "txt":
            return font_size
        if doc_type == "md":
            return font_size * 2
        if doc_type == "docx":
            return font_size * 3
        raise ValueError("invalid doc type")
    return inner_function

In [2]:
run_cases = [
    (12, "txt", 12),
    (16, "md", 32),
]

submit_cases = run_cases + [
    (14, "html", "invalid doc type"),
    (0, "txt", 0),
    (50, "md", 100),
]


def test(input1, input2, expected_output):
    print("---------------------------------")
    print(f"Inputs:")
    print(f" * font_size: {input1}")
    print(f" * doc_type: {input2}")
    print(f"Expected: {expected_output}")
    try:
        result = converted_font_size(input1)(input2)
    except Exception as error:
        result = str(error)
    print(f"Actual:   {result}")
    if result == expected_output:
        print("Pass")
        return True
    print("Fail")
    return False


def main():
    passed = 0
    failed = 0
    skipped = len(submit_cases) - len(test_cases)
    for test_case in test_cases:
        correct = test(*test_case)
        if correct:
            passed += 1
        else:
            failed += 1
    if failed == 0:
        print("============= PASS ==============")
    else:
        print("============= FAIL ==============")
    if skipped > 0:
        print(f"{passed} passed, {failed} failed, {skipped} skipped")
    else:
        print(f"{passed} passed, {failed} failed")


test_cases = submit_cases
if "__RUN__" in globals():
    test_cases = run_cases

main()

---------------------------------
Inputs:
 * font_size: 12
 * doc_type: txt
Expected: 12
Actual:   12
Pass
---------------------------------
Inputs:
 * font_size: 16
 * doc_type: md
Expected: 32
Actual:   32
Pass
---------------------------------
Inputs:
 * font_size: 14
 * doc_type: html
Expected: invalid doc type
Actual:   invalid doc type
Pass
---------------------------------
Inputs:
 * font_size: 0
 * doc_type: txt
Expected: 0
Actual:   0
Pass
---------------------------------
Inputs:
 * font_size: 50
 * doc_type: md
Expected: 100
Actual:   100
Pass
============= PASS ==============
5 passed, 0 failed


**Currying** is useful when it is necessary to change a function signature to make it conform to a specific shape:

In [ ]:
#Suposse we have a colorize functions that accepts a function and a document:
def colorize(converter,doc):
    #...
    converter(doc)
    #...
#Suposse this other function that is wished to pass to colorize function
def md_to_html(doc, asterisk_style):
    #...
'''
It is not possible to pass md_to_html directly to colorize, because the 1st function 
requires two arguments. So the function should be curried:
'''
def md_to_html(asterisk_style):
    def asterisk_md_to_html(doc):
        #...
    return asterisk_md_to_html

#And now, the function can be passed to colorize to work over the same variable
md_to_html_italic = md_to_html('italic')
colorize(md_to_html_italic, doc)
#This is the same than writing
colorize(md_to_html('italic'), doc)
#And when calling the converter inside colorize, it have a this structure:
md_to_html('italic')(doc)
#But colorize is ignorant about what the converter does. This is know as black box

This is an example of using **currying** to calculate the amount of lines that have a certain amount of a str, or sequence of characters

In [4]:
def lines_with_sequence(char):
    def with_char(length):
        sequence = char * length
        def with_lenght(doc):
            number_of_lines_with_sequence = 0
            for line in doc.split():
                if sequence in line:
                    number_of_lines_with_sequence += 1
            return number_of_lines_with_sequence
        return with_lenght
    return with_char

In [5]:
run_cases = [
    (
        "#",
        3,
        """###
@##
$$$
###""",
        2,
    ),
    (
        "$",
        2,
        """$$$
$
***
@@@
$$
$$$""",
        3,
    ),
]

submit_cases = run_cases + [
    ("%", 1, "", 0),
    (
        "*",
        3,
        """***
*
$$$$$$
xxx
****
***
***""",
        4,
    ),
]


def test(char, length, doc, expected_output):
    print("---------------------------------")
    print(f"Input char: {char}")
    print(f"Input length: {length}")
    print(f"Input doc:")
    print(doc)
    print(f"Expected: {expected_output}")
    num_lines = lines_with_sequence(char)(length)(doc)
    print(f"Actual:   {num_lines}")
    if num_lines == expected_output:
        print("Pass")
        return True
    print("Fail")
    return False


def main():
    passed = 0
    failed = 0
    skipped = len(submit_cases) - len(test_cases)
    for test_case in test_cases:
        correct = test(*test_case)
        if correct:
            passed += 1
        else:
            failed += 1
    if failed == 0:
        print("============= PASS ==============")
    else:
        print("============= FAIL ==============")
    if skipped > 0:
        print(f"{passed} passed, {failed} failed, {skipped} skipped")
    else:
        print(f"{passed} passed, {failed} failed")


test_cases = submit_cases
if "__RUN__" in globals():
    test_cases = run_cases

main()

---------------------------------
Input char: #
Input length: 3
Input doc:
###
@##
$$$
###
Expected: 2
Actual:   2
Pass
---------------------------------
Input char: $
Input length: 2
Input doc:
$$$
$
***
@@@
$$
$$$
Expected: 3
Actual:   3
Pass
---------------------------------
Input char: %
Input length: 1
Input doc:

Expected: 0
Actual:   0
Pass
---------------------------------
Input char: *
Input length: 3
Input doc:
***
*
$$$$$$
xxx
****
***
***
Expected: 4
Actual:   4
Pass
============= PASS ==============
4 passed, 0 failed


Another example using syntax from Markdown to display images

In [46]:
def create_markdown_image(alt_text):
    alt_text = f"![{alt_text}]"
    def url_encoder(url):
        #Encoding sequences of specific parentheses
        url = url.replace("(","%28")
        url = url.replace(")","%29")
        def define_title_optional(title=None):
            if title != None:
                return f'{alt_text}({url} "{title}")'
            else:
                return f'{alt_text}({url})'
        return define_title_optional
    return url_encoder

In [47]:
run_cases = [
    (
        "seal",
        "https://imgur.com/oglPAXK",
        "this is a seal",
        '![seal](https://imgur.com/oglPAXK "this is a seal")',
    ),
    (
        "cinnamon roll",
        "https://imgur.com/a/0MyOP",
        "this is a cinnamon roll",
        '![cinnamon roll](https://imgur.com/a/0MyOP "this is a cinnamon roll")',
    ),
]

submit_cases = run_cases + [
    (
        "banana",
        "https://imgur.com/nlArAKx",
        None,
        "![banana](https://imgur.com/nlArAKx)",
    ),
    (
        "not an image",
        "https://en.wikipedia.org/wiki/Variable_(computer_science)",
        "showing escape characters",
        '![not an image](https://en.wikipedia.org/wiki/Variable_%28computer_science%29 "showing escape characters")',
    ),
]


def test(alt_text, url, title, expected_output):
    print("---------------------------------")
    print(f"Inputs:")
    print(f"* Alt Text: {alt_text}")
    print(f"* URL: {url}")
    print(f"* Title: {title}")
    print(f"Expected: {expected_output}")
    result = create_markdown_image(alt_text)(url)()
    if title:
        result = create_markdown_image(alt_text)(url)(title)
    print(f"Actual:   {result}")
    if result == expected_output:
        print("Pass")
        return True
    print("Fail")
    return False


def main():
    passed = 0
    failed = 0
    skipped = len(submit_cases) - len(test_cases)
    for test_case in test_cases:
        correct = test(*test_case)
        if correct:
            passed += 1
        else:
            failed += 1
    if failed == 0:
        print("============= PASS ==============")
    else:
        print("============= FAIL ==============")
    if skipped > 0:
        print(f"{passed} passed, {failed} failed, {skipped} skipped")
    else:
        print(f"{passed} passed, {failed} failed")


test_cases = submit_cases
if "__RUN__" in globals():
    test_cases = run_cases

main()

---------------------------------
Inputs:
* Alt Text: seal
* URL: https://imgur.com/oglPAXK
* Title: this is a seal
Expected: ![seal](https://imgur.com/oglPAXK "this is a seal")
Actual:   ![seal](https://imgur.com/oglPAXK "this is a seal")
Pass
---------------------------------
Inputs:
* Alt Text: cinnamon roll
* URL: https://imgur.com/a/0MyOP
* Title: this is a cinnamon roll
Expected: ![cinnamon roll](https://imgur.com/a/0MyOP "this is a cinnamon roll")
Actual:   ![cinnamon roll](https://imgur.com/a/0MyOP "this is a cinnamon roll")
Pass
---------------------------------
Inputs:
* Alt Text: banana
* URL: https://imgur.com/nlArAKx
* Title: None
Expected: ![banana](https://imgur.com/nlArAKx)
Actual:   ![banana](https://imgur.com/nlArAKx)
Pass
---------------------------------
Inputs:
* Alt Text: not an image
* URL: https://en.wikipedia.org/wiki/Variable_(computer_science)
* Title: showing escape characters
Expected: ![not an image](https://en.wikipedia.org/wiki/Variable_%28computer_scien

In [56]:
def new_resizer(max_width, max_height):
    def min_size(min_width = 0,min_height = 0):
        if min_width > max_width or min_height > max_height:
            raise Exception("minimum size cannot exceed maximum size")
        def resizer_final(width, height):
            width = min(max(width,min_width), max_width)
            height = min(max(height,min_height), max_height)
            return width, height
        return resizer_final
    return min_size

In [57]:
run_cases = [
    (
        (1920, 1080),
        (800, 600),
        [
            (
                (2560, 1440),
                (1920, 1080),
            ),
            (
                (500, 400),
                (800, 600),
            ),
            (
                (1600, 1000),
                (1600, 1000),
            ),
            (
                (800, 600),
                (800, 600),
            ),
            (
                (1920, 1080),
                (1920, 1080),
            ),
        ],
        None,
    ),
    (
        (1200, 800),
        (1600, 800),
        [],
        "minimum size cannot exceed maximum size",
    ),
    (
        (1600, 800),
        (1200, 1200),
        [],
        "minimum size cannot exceed maximum size",
    ),
]

submit_cases = run_cases + [
    (
        (1600, 1200),
        (1200, 800),
        [
            (
                (1601, 799),
                (1600, 800),
            ),
            (
                (1199, 1201),
                (1200, 1200),
            ),
        ],
        None,
    ),
    (
        (600, 600),
        (600, 600),
        [
            (
                (601, 601),
                (600, 600),
            ),
            (
                (599, 599),
                (600, 600),
            ),
        ],
        None,
    ),
    (
        (100, 100),
        (),
        [
            (
                (200, 200),
                (100, 100),
            ),
            (
                (0, 0),
                (0, 0),
            ),
        ],
        None,
    ),
]


def test(max_size, min_size, image_sizes, expected_error):
    print("---------------------------------")
    print(f"Max Size:  {max_size}")
    if min_size:
        print(f"Min Size:  {min_size}")
    try:
        resize_image = new_resizer(*max_size)(*min_size)
    except Exception as e:
        print(f"Expected Error: {expected_error}")
        print(f"  Actual Error: {str(e)}")
        if str(e) == expected_error:
            print("Pass")
            return True
        print("Fail")
        return False
    if expected_error is not None:
        print(f"Expected Error: {expected_error}")
        print("Fail")
        return False
    print("Resizing Images...")
    failed = False
    for size in image_sizes:
        result = resize_image(*size[0])
        print(f" * Image Size: {size[0]}")
        print(f" *   Expected: {size[1]}")
        print(f" *     Actual: {result}")
        if result == size[1]:
            print("Pass")
        else:
            print("Fail")
            print(result)
            print(size[1])
            failed = True
    passed = not failed
    return passed


def main():
    passed = 0
    failed = 0
    skipped = len(submit_cases) - len(test_cases)
    for test_case in test_cases:
        correct = test(*test_case)
        if correct:
            passed += 1
        else:
            failed += 1
    if failed == 0:
        print("============= PASS ==============")
    else:
        print("============= FAIL ==============")
    if skipped > 0:
        print(f"{passed} passed, {failed} failed, {skipped} skipped")
    else:
        print(f"{passed} passed, {failed} failed")


test_cases = submit_cases
if "__RUN__" in globals():
    test_cases = run_cases

main()

---------------------------------
Max Size:  (1920, 1080)
Min Size:  (800, 600)
Resizing Images...
 * Image Size: (2560, 1440)
 *   Expected: (1920, 1080)
 *     Actual: (1920, 1080)
Pass
 * Image Size: (500, 400)
 *   Expected: (800, 600)
 *     Actual: (800, 600)
Pass
 * Image Size: (1600, 1000)
 *   Expected: (1600, 1000)
 *     Actual: (1600, 1000)
Pass
 * Image Size: (800, 600)
 *   Expected: (800, 600)
 *     Actual: (800, 600)
Pass
 * Image Size: (1920, 1080)
 *   Expected: (1920, 1080)
 *     Actual: (1920, 1080)
Pass
---------------------------------
Max Size:  (1200, 800)
Min Size:  (1600, 800)
Expected Error: minimum size cannot exceed maximum size
  Actual Error: minimum size cannot exceed maximum size
Pass
---------------------------------
Max Size:  (1600, 800)
Min Size:  (1200, 1200)
Expected Error: minimum size cannot exceed maximum size
  Actual Error: minimum size cannot exceed maximum size
Pass
---------------------------------
Max Size:  (1600, 1200)
Min Size:  (120